In [1]:
import os
import re
import pandas as pd
from collections import defaultdict


### Functions for parsing the yw models file. 

In [2]:
### function extracting the programs.

def extract_steps(line):
    
    #% FACT: program(program_id, program_name, qualified_program_name, begin_annotation_id, end_annotation_id).
    #print("Extracting steps.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    
    data = data.split(',')
    #print(data)
    steps[data[0]] = {
                      "program_name" :data[1].strip().strip("'").strip('"'),
                      "qualified_program_name": data[2].strip().strip("'").strip('"')
                    }
    
    #print(steps)
    return

In [3]:
### function extracting the workflow.

def extract_workflows(line):
    
    #print(line)
    data = line[line.index("(")+1:line.index(")")]
    #print(data)
    workflow[data] = steps[data]['program_name']
       
    return 

In [4]:
### function extracting the ports

def extract_ports(line):
    #print("Extracting ports.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    
    #% FACT: port(port_id, port_type, port_name, qualified_port_name, port_annotation_id, data_id).
    
    data = data.split(',')
    
    ports[data[0]] ={
                     "port_type": data[1].strip().strip("'").strip('"').upper(),
                     "port_name": data[2].strip().strip("'").strip('"'),
                     "qualified_port_name" : data[3].strip().strip("'").strip('"'), 
                     "data_id":data[5].strip().strip("'").strip('"')
                    }
    return 

In [5]:
### function extracting the input ports

def input_ports(line,in_ports):
    #print("Extracting Input ports.")
    
    ## Extract data between "(" and ")".
    ## % FACT: has_in_port(step_id, port_id).

    data = line[line.index("(")+1:line.index(")")]
    
    data = data.split(',')
    
    in_ports[data[0]].append((data[0].strip(), data[1].strip()))
    
    return

In [6]:
### function extracting the output ports 

def output_ports(line,out_ports): 
    #print("Extracting output ports.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    out_ports[data[0]].append((data[0].strip(), data[1].strip()))
    #print(data)
    return

In [7]:
### function extracting the output ports 

def get_port_data(line): 
    #print("Extracting output ports.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    port_data[data[0]]= { 
                          "data_name" : data[1].strip(), 
                          "qual_data_name" : data[2].strip()
                        } 
    #print(data)
    return

In [8]:
### function extracting the output ports 

def get_channel(line): 
    #print("Extracting output ports.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    channel[data[0]]= data[1].strip()
    #print(data)
    return

In [9]:
### function extracting the output ports 

def get_port_connects_to_channel(line): 
     
    data = line[line.index("(")+1 : line.index(")")]
    data = data.split(',')
    port_connects_to_channel[data[0]]= data[1].strip()
    #print(data)
    return

In [10]:
def get_inflow_connections(line): 
     
    #print(line)
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    inflow_conn[data[0]]= data[1].strip()
    #print(data)
    return

In [11]:
def get_outflow_connections(line): 
    #print("Extracting output ports.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    outflow_conn[data[0]]= data[1].strip()
    #print(data)
    return 

In [12]:
### function extracting the subprograms 

def has_subprogram(line): 
    #print("Extracting output ports.")
    ## Extract data between "(" and ")".
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    list1 = []
    #print(data[0],data[1])
    if data[0] in sub_programs.keys():
        list1 = sub_programs[data[0]]
        list1.append(data[1].strip())
        sub_programs[data[0]] = list1
    else:
        list1.append(data[1].strip())
        sub_programs[data[0]]= list1
    #print(data)
    return

In [13]:
### function extracting the port alias

def port_alias(line):
    data = line[line.index("(")+1:line.index(")")]
    data = data.split(',')
    port_id = data[0].strip()
    alias = data[1].strip().strip("'")
    
    qualified_name = ports[port_id]['qualified_port_name']
    port_name = ports[port_id]['port_name'] 
    
    port_alias_name[port_id] = alias
    
    #print(port_name, port_id,qualified_name)
    
    ### regex for splitting the string and getting qualified program name
    regex = re.compile(r'>|<')
    qname = regex.split(qualified_name)
    
    pname = qname[0].split('.')[-1] 
    
    if pname.find('-') > -1:
        port_alt_name[alias] = pname[:-1]+ '/' +port_name
    else:
        port_alt_name[alias] = pname + '/' +port_name
        
    return 



In [14]:
## Method for getting the input ports 
## for a specific program 

def get_in_ports(program_id):
    #print("inside in ports")
    input_ports =[]
    for s in in_ports[program_id]:
        #print(s)
        input_ports.append(ports[s[1]]['port_name'])
        #print(ports[int(s[1])]['port_name'])
    return input_ports

In [15]:
## Method for getting the output ports 
## for a specific program 

def get_out_ports(program_id):
    output_ports =[]
    for s in out_ports[program_id]:
        output_ports.append(ports[s[1]]['port_name'])
    return output_ports

In [16]:
## Method for create a cwl file 
## It has the required header and formatting

def create_file(filename, program_id):
    header = '''
#!/usr/bin/env cwl-runner


cwlVersion: v1.0
class: CommandLineTool
'''
    with open(filename, "w+") as writer:
        input_buffer = 'inputs: \n'
        inp_temp = []
        out_temp = []
        for p in get_in_ports(program_id):
            inp_temp.append(p)
            input_buffer = input_buffer + ' '+ p + ': \n'+ '  type: string \n \n'

        #print(input_buffer)
        output_buffer = 'outputs: \n'
        output_buffer.strip()

        for p in get_out_ports(program_id):
            #print(len(p))
            if len(p)==0 : 
                print("No output ports")
            else:
                output_buffer = output_buffer + ' '+ p + ': \n'+ '  type: string \n \n'
            out_temp.append(p)

        writer.write(header)    
        writer.write(input_buffer)
        writer.write(output_buffer)
        writer.write("\n")
        writer.write("baseCommand: ")
        cwl_file_data [program_id]= {"program_name" : steps[program_id]["program_name"],
                                     "input_ports"  : inp_temp,
                                     "output_ports" : out_temp
                                    }
    return

In [17]:
### This function passes file name 
### for creating cwl files. 

def create_cwl_files(program_id,dir_name): 
    pname= steps[program_id]['program_name']
    filename = dir_name + pname + ".cwl"
    create_file(filename, program_id)
    
    return 

In [18]:
### Function for exrtacting the steps in a wf. 

def get_wf_steps(wf_id, wf_name):
    
    wf_steps = "steps: \n " 
    
    ## first get the sub-programs of a workflow: 
    for s in sub_programs[wf_id]:
        prog_name= steps[s]["program_name"]
        step_outports =[]
        if s in workflow.keys():
            filename ='wf_'+ prog_name + '.cwl'
        else:
            filename =  prog_name + '.cwl'
            
        step_buffer = prog_name + ': \n ' + ' run: ' + filename + ' \n ' + ' in: \n'
        
        inp_temp = []
        for pid in in_ports[s]:
            out_in_ports(s)
            pname, qual_pname= get_inports_subprograms(pid[1], wf_id)
            print(pname, qual_pname, pid[1])
            inp_temp.append((pname,qual_pname))
            step_buffer = step_buffer + '   ' + pname + ': ' + qual_pname + '\n' 
            '''
            Get the outports of the program.
            '''
            
            step_outports= get_out_ports(s)
            if len(step_outports) == 0: 
                print("The step " + prog_name + " doesn't have output ports. CWL model will fail.")
                
        step_buffer =  step_buffer + "  out: [" + ' , '.join(step_outports) + '] \n ' 
        
        cwl_wf_data[s]  = {         
                            "program_name" : filename,
                            "input_ports" : inp_temp,
                            "output_ports": step_outports
                          }
        wf_steps = wf_steps + step_buffer
    
    return wf_steps

In [19]:
def get_set_outports(program_id):
    set_outports = {}
    for p in steps.keys():
        #print(p)
        for o in out_ports[str(p)]:
            set_outports[ports[o[1]]['data_id']] = p
    
    #print(set_outports)
    return set_outports

In [20]:
def out_in_ports(program_id):
    
    #print(steps[program_id]["program_name"])
    for ip in in_ports[program_id]:
        op_set = get_set_outports(program_id)
        #print(op_set)
        data_id = ports[ip[1]]['data_id']
        common_data_id = set([data_id]).intersection(set(op_set.keys()))
      
    ## Check if input port dataid is an output port 
    ## dataid of some previous or same step
    
        if common_data_id:
            pid = list(common_data_id)[0]
            prid = op_set[pid] 
            prog_name= steps[str(prid)]["program_name"] +'/'
            #print(prog_name)
            #if program_id in sub_programs.keys():
            #    prog_name=''

            for o in out_ports[str(prid)]:
                if ports[o[1]]["data_id"] == pid:
                    port_name = ports[o[1]]["port_name"]
                    out_in_port[data_id]= prog_name + port_name
            
            '''
            if pid in port_alias_name.keys():
                port_name = ports[pid]["port_name"]
            else:
                port_name = port_data[pid]["data_name"].strip("'")
                
            out_in_port[data_id]= prog_name +'/'+ port_name
            '''

In [85]:
def get_qual_port_name(port_name):
    regex = re.compile(r'>|<')
    for port in ports:
        #print(ports[port]["port_name"])
        if ports[port]["port_name"] == port_name:
            
            temp = regex.split(ports[port]['qualified_port_name'])
            step_name= temp[0].split('.')[-1]
            qname = step_name[:-1]
            if qname == steps['1']["program_name"] and str(temp).find('.') > -1:          
                return qname
            elif qname != steps['1']["program_name"]:
                return qname 
        #else:
        #    return "Not in ports list"

In [86]:
## Get the inports of the subprograms

def get_inports_subprograms(port_id,wf_id):
    
    qualified_port=''
    pname = ports[port_id]["port_name"]
    data_id = ports[port_id]["data_id"]
    #print(pname)
    a = set([port_id]).intersection(set(list(port_alias_name.keys())))
    #print(a)

    if data_id in out_in_port.keys():
        qualified_port = out_in_port[data_id]
    elif pname in get_in_ports(wf_id):
        qualified_port = pname
    #elif pname in in_out_ports.keys():
    #    qualified_port = in_out_ports[pname][1]
    elif a:
        for e in a:
            #print(port_alias_name[e] +'Alias name' )
            if port_alias_name[e] in get_in_ports(wf_id):
                qualified_port = port_alias_name[e]
            else:
                qualified_port = get_qual_port_name(port_alias_name[e])
    else:
        qualified_port = "Parameter"
        
    return pname, qualified_port 

In [87]:
def get_wf_outports(port_id):
    
    pname = ports[port_id]["port_name"]   

    ch_id = outflow_conn[port_id]
    #conn_pid = 
    for p in port_connects_to_channel:
        if port_connects_to_channel[p] == ch_id:
            conn_pid = p
    port_name = ports[conn_pid]["port_name"]
    #print(port_name)
    qname = get_qual_port_name(port_name) + '/' + port_name


    return pname, qname

def get_in_out_ports():
    for sid in steps.keys(): 
        for id in range(0,len(steps)+1):
            a = set(get_in_ports(str(id))).intersection(set(get_out_ports(str(sid))))
            #print(a)
            if len(a) > 0 : 
                for elem in a: 
                    #print(steps[str(id)]["program_name"] , "\t\t", steps[sid]["program_name"]+"/"+ elem )
                    in_out_ports[elem] = [steps[str(id)]["program_name"], steps[sid]["program_name"]+"/"+ elem]
                

## function for getting the ports with the program name
def link_wf_ports(workflow_id):
    wf_out_ports=[]
    
    for sp in sub_programs[workflow_id]:
        for p in set(get_out_ports(workflow_id)).intersection(set(get_out_ports(sp))):
            wf_out_ports.append(steps[sp]['program_name']+ '/' + p)
    
    return wf_out_ports

In [88]:
def workflow_detail(program_id, wf_name):
    header = '''
#!/usr/bin/env cwl-runner
cwlVersion: v1.0
class: Workflow
requirements:
    - class: SubworkflowFeatureRequirement
'''
    input_buffer = 'inputs: \n'
    inp_temp = []
    out_temp = []
    for p in get_in_ports(program_id):
        inp_temp.append((p,p))
        input_buffer = input_buffer.strip('') + '  '+ p + ': \n'+ '   type: string \n'
        
    output_buffer = 'outputs: \n'
    #out_wf_ports = link_wf_ports(program_id)
    
    for port in out_ports[program_id]:
        #print(port)
        pname, qname = get_wf_outports(port[1])
        out_temp.append((pname,qname))
        output_buffer = output_buffer + '  ' +  pname  + ': \n' + '   type: string \n   outputSource: '+ qname +'\n'


    wf_step = get_wf_steps(program_id, wf_name)
    
    cwl_wf_data[program_id]  = {"program_name" : steps[program_id]["program_name"],
                                  "input_ports" : inp_temp,
                                  "output_ports": out_temp
                               }
    #print(wf_step)
    return header, input_buffer, output_buffer, wf_step 

In [89]:
def create_wf_file(filename, program_id,wf_name):
    #filename = 'wf_' + steps['1']["program_name"] + '.cwl'

    with open(filename, 'w+') as file_writer:
        hdr, inp_buff, out_buff, wf_step = workflow_detail(program_id,wf_name)
        file_writer.write(hdr)
        file_writer.write(inp_buff)
        file_writer.write(out_buff)
        file_writer.write(wf_step)

In [90]:
def read_yw_model(filename):
    regex = re.compile(r'^%')
    chk_channel = re.compile(r'^ch')
    with open(filename, "r") as yw_struct:
        line = yw_struct.readline()
        while line:
            #print(line)
            comments = regex.match(line)
            if comments: 
                #print(line)
                line = yw_struct.readline()
            else:  
                if("program(" in line and "has_subprogram" not in line):
                    extract_steps(line)
                elif("workflow(" in line):
                    extract_workflows(line)            
                elif("has_subprogram(" in line):
                    has_subprogram(line)            
                elif("port(" in line  and "has_in_port(" not in line and "has_out_port" not in line):
                #    print(line)
                    extract_ports(line)
                elif("has_in_port(" in line):
                    input_ports(line,in_ports)
                elif("has_out_port(" in line ):
                    output_ports(line,out_ports)
                elif ("port_alias(" in line ): 
                    port_alias(line)
                elif("data(" in line):
                    get_port_data(line)
                elif("port_connects_to_channel(" in line):
                    #print(line)
                    get_port_connects_to_channel(line)
                elif("inflow_connects_to_channel("in line):
                    #print(line)
                    get_inflow_connections(line)                    
                elif("outflow_connects_to_channel("in line):
                    get_outflow_connections(line)
                elif( chk_channel.match(line) ):
                    get_channel(line)

                line = yw_struct.readline()
        
        #get_in_out_ports()

In [91]:
steps = {}
ports = {}
workflow = {}
in_ports = defaultdict(list)
out_ports = defaultdict(list)
sub_programs = {}
port_alias_name= {}
port_alt_name={}
in_out_ports={}
out_in_port={}
cnt =0

qualified_port=''
channel={}
port_data={}
port_connects_to_channel={}
inflow_conn={}
outflow_conn={}
cwl_file_data = {}
cwl_wf_data = {}

yw_model_file = "Examples/paleocar_models/paleocar_models.P"

read_yw_model(yw_model_file)
for w in sorted(workflow, reverse=True):
    dir_name = 'Examples/' + steps['1']["program_name"] + '/'
    
    wf_name = steps[w]["program_name"]
    
    #print(w)
    '''
        Get the steps name and generate the files
        for those steps.
    '''
    for sp in sub_programs[w]:
        #print(sp)
        create_cwl_files(sp,dir_name)
    '''
        Get the workflow name and generate the files
        for those workflows.
    '''
    filename =  dir_name + 'wf_' + steps[w]["program_name"] + '.cwl'
    #print(filename)
    create_wf_file(filename,w, wf_name)
    
    df_file = pd.DataFrame.from_dict(cwl_file_data,orient='index')
    df_wf = pd.DataFrame.from_dict(cwl_wf_data, orient='index')
    df_file.to_csv(dir_name + wf_name + '.csv')
    df_wf = pd.DataFrame.from_dict(cwl_wf_data, orient='index')
    df_wf.to_csv(dir_name +'wf_'+ wf_name +  '.csv')

predlist predlist 27
carscores carscores 28
max_preds max_preds 29
models defineLinearModels/models 32
matches defineLinearModels/matches 33
coefficients calculateLinearModels/coefficients 36
model_errors calculateLinearModels/model_errors 37
verbose verbose 8
The step print_message doesn't have output ports. CWL model will fail.
itrdb itrdb 9
calibration_years calibration_years 10
min_width min_width 11
itrdb itrdb 14
prediction_years prediction_years 15
min_width min_width 16
reconstruction_matrix get_reconstruction_matrix/reconstruction_matrix 18
prism_data_for_coordinates prism_data_for_coordinates 20
predictor_matrix get_predictor_matrix/predictor_matrix 21
predlist get_predlist/predlist 23
carscores get_carscores/carscores 24
max_preds get_predictor_matrix/max_preds 25
linear_models calculate_Models/linear_models 39


In [92]:
df_wf.head()

,program_name,input_ports,output_ports
1,paleocar_models,"[(prediction_years, prediction_years), (prism_...","[(paleocar_models, optimizeModels/paleocar_mod..."
10,simplifyLinearModels.cwl,"[(coefficients, calculateLinearModels/coeffici...",[final_models]
11,optimizeModels.cwl,"[(linear_models, calculate_Models/linear_models)]",[paleocar_models]
2,print_message.cwl,"[(verbose, verbose)]",[]
3,get_predictor_matrix.cwl,"[(itrdb, itrdb), (calibration_years, calibrati...","[predictor_matrix, max_preds]"


In [106]:
def workflow_content():
    
    for s in steps: 
        print(df_wf.loc[s]['program_name'])
        #get_input_ports(df_wf.loc[s]['input_ports'])
        get_output_ports(df_wf.loc[s]['output_ports'])
    return  

In [107]:
def get_input_ports(input_port_list):
    
    for i in input_port_list:
        #print(i)
        pname , qual_pname = i
        #print(pname , qual_pname)
    return 

In [114]:
def get_output_ports(output_port_list):
    
    if output_port_list == None: 
        print ("No output ports.")
    else:
        for o in output_port_list:
            #print(o)
            pname , qual_pname = o
            #print(pname , qual_pname)
    return 

In [115]:
workflow_content()

paleocar_models
print_message.cwl
get_predictor_matrix.cwl


ValueError: too many values to unpack (expected 2)

In [49]:
df_wf.loc['2']['output_ports'] = [('cwl_out_', df_wf.loc['2']['program_name'][:-4])]

In [58]:
ops.append(df_wf.loc['2']['output_ports'])

In [60]:
df_wf.loc['1']['output_ports'] = ops

In [63]:
df_wf['output_ports']

1     [(paleocar_models, Not in ports list/paleocar_...
10                                       [final_models]
11                                    [paleocar_models]
2                           [(cwl_out_, print_message)]
3                         [predictor_matrix, max_preds]
4                               [reconstruction_matrix]
5                                            [predlist]
6                                           [carscores]
7                                       [linear_models]
8                                     [models, matches]
9                          [coefficients, model_errors]
Name: output_ports, dtype: object